In [74]:
import sys
import os
from scipy.io import wavfile
import numpy as np
data_folder = "/Volumes/CB_RESEARCH/vocal_synthesis/"
import theano
from theano import tensor as T
import lasagne
from lasagne.layers import *
from lasagne.init import *
from lasagne.nonlinearities import *
import cPickle as pickle
import experiment

In [75]:
with open("out.model") as f:
    model = pickle.load(f)

In [76]:
args = dict()
args["batch_size"] = 128
args["chunk_size"] = 200
args["learning_rate"] = 0.01
args["momentum"] = 0.9
args["num_epochs"] = 20

symbols = experiment.prepare(args)
l_out = symbols["l_out"]

Number of params in model: 20201


In [78]:
model

{'max': 9124.0, 'min': -8936.0, 'params': [W, b, W, b]}

In [81]:
tmp_model = [ model["params"][0].get_value(), model["params"][1].get_value(),
           model["params"][2].get_value(), model["params"][3].get_value() ]

In [82]:
lasagne.layers.set_all_param_values(l_out, tmp_model)

In [96]:
fs, data = wavfile.read(data_folder + os.path.sep + "data.wav")
data = np.asarray(data, dtype="float32")
print "min, max before normalisation: %f, %f\n" % (np.min(data), np.max(data)) 
data = (data - model["min"]) / (model["max"] - model["min"])
print "min, max after normalisation: %f, %f\n" % (np.min(data), np.max(data)) 
data = data.tolist()

min, max before normalisation: -8936.000000, 9124.000000

min, max after normalisation: 0.000000, 1.000000



In [86]:
get_out = symbols["get_out"]

In [87]:
seed_vector = data[5000:5000+200]
#get_out( np.asarray([data[0:200]], dtype="float32") )
print np.min(seed_vector)
print np.max(seed_vector)

0.455647826195
0.535382032394


In [88]:
for b in range(0, 100000):
    next_value = get_out( np.asarray([ seed_vector[b::] ], dtype="float32") )[0][0]
    #print next_value
    seed_vector.append(next_value)

In [89]:
print np.min(seed_vector)
print np.max(seed_vector)

0.449085719942
0.584380835989


In [90]:
len(seed_vector) / fs

6

In [91]:
result = (np.asarray(seed_vector) * (model["max"] - model["min"])) + model["min"]
result = np.asarray(result, dtype="int32")

In [92]:
result

array([419, 388, 336, ...,  79,  79,  79], dtype=int32)

In [93]:
print np.min(result)
print np.max(result)

-825
1617


In [94]:
print model["min"]
print model["max"]

-8936.0
9124.0


In [73]:
wavfile.write(filename="/tmp/file.wav", rate=fs, data=result)

In [36]:
print np.min(result)
print np.max(result)

-707
733


In [37]:
print np.min(tmp)
print np.max(tmp)

-8936
9124


In [39]:
(505.0 - (-8936)) / (9124 - -8936)

0.5227574750830565

In [98]:
_, tmpdata = wavfile.read(data_folder + os.path.sep + "data.wav")
tmpdata = np.asarray(tmpdata, dtype="float32")
tmpdata

array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32)

In [99]:
tmpdata2 = (tmpdata - model["min"]) / (model["max"] - model["min"])

In [100]:
np.min(tmpdata2), np.max(tmpdata2)

(0.0, 1.0)